In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pandas as pd

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc) 

In [2]:
spark=SparkSession.builder.getOrCreate()

In [3]:
# now not pandas but SPARK needs to read the files

h_cdr = spark.read.csv("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\cdr.csv", header = True)
h_complaint = spark.read.csv("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\complaint.csv", header = True)
h_cust_churn = spark.read.csv("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\customer_churn_demography.csv", header = True)
h_revenue = spark.read.csv("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\revenue.csv", header = True)
h_signal = spark.read.csv("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\signalstrength.csv", header = True)

In [4]:
h_df1 = (h_cust_churn.join(h_cdr, "phone number")
        .join(h_complaint, "phone number")
        .join(h_revenue, "phone number")
        .join(h_signal, "phone number"))

In [5]:
from pyspark.sql.types import *
h_final = h_df1.withColumn("account length", h_df1["account length"].cast(FloatType()))\
.withColumn("account (days)", h_df1["account (days)"].cast(FloatType()))\
.withColumn("Age", h_df1["Age"].cast(FloatType()))\
.withColumn("%call drops", h_df1["%call drops"].cast(FloatType()))\
.withColumn("%Packet_drops", h_df1["%Packet_drops"].cast(FloatType()))\
.withColumn("number vmail messages", h_df1["number vmail messages"].cast(FloatType()))\
.withColumn("total day minutes", h_df1["total day minutes"].cast(FloatType()))\
.withColumn("total day calls", h_df1["total day calls"].cast(FloatType()))\
.withColumn("total eve minutes", h_df1["total eve minutes"].cast(FloatType()))\
.withColumn("total eve calls", h_df1["total eve calls"].cast(FloatType()))\
.withColumn("total night minutes", h_df1["total night minutes"].cast(FloatType()))\
.withColumn("total night calls", h_df1["total night calls"].cast(FloatType()))\
.withColumn("total intl minutes", h_df1["total intl minutes"].cast(FloatType()))\
.withColumn("total intl calls", h_df1["total intl calls"].cast(FloatType()))\
.withColumn("customer service calls", h_df1["customer service calls"].cast(FloatType()))\
.withColumn("Number of complaint raised", h_df1["Number of complaint raised"].cast(FloatType()))\
.withColumn("Time for resolution", h_df1["Time for resolution"].cast(FloatType()))\
.withColumn("payment_made_but_order_not_placed_and_amount_is_deducted_from_my_bank_account", h_df1["payment_made_but_order_not_placed_and_amount_is_deducted_from_my_bank_account"].cast(FloatType()))\
.withColumn("Billing_issue", h_df1["Billing_issue"].cast(FloatType()))\
.withColumn("Calls_are_not_going", h_df1["Calls_are_not_going"].cast(FloatType()))\
.withColumn("drop_calls", h_df1["drop_calls"].cast(FloatType()))\
.withColumn("No_internet_connection", h_df1["No_internet_connection"].cast(FloatType()))\
.withColumn("order", h_df1["order"].cast(FloatType()))\
.withColumn("payment_refund", h_df1["payment_refund"].cast(FloatType()))\
.withColumn("recharge_offer_service_fraud", h_df1["recharge_offer_service_fraud"].cast(FloatType()))\
.withColumn("Slow_network_coverage", h_df1["Slow_network_coverage"].cast(FloatType()))\
.withColumn("uninformed___unsubscribed_weekly_amount_deduction", h_df1["uninformed___unsubscribed_weekly_amount_deduction"].cast(FloatType()))\


In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

In [32]:

h_df2= h_final.select('%call drops', 'total day minutes','total intl calls','customer service calls')

In [23]:
data_train_rdd=h_df2.rdd

In [9]:
v1=data_train_rdd.map(lambda x:x[1])

In [10]:
v2=data_train_rdd.map(lambda x:x[2])

In [30]:
from pyspark.mllib.stat import Statistics
NewVector=data_train_rdd.map(lambda x:x[0:6]) 

In [31]:
correlation_matrix = Statistics.corr(NewVector, method="pearson")

Py4JJavaError: An error occurred while calling o388.corr.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 29, localhost, executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:109)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:153)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:122)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.api.python.PythonRDD$.org$apache$spark$api$python$PythonRDD$$write$1(PythonRDD.scala:192)
	at org.apache.spark.api.python.PythonRDD$$anonfun$writeIteratorToStream$1.apply(PythonRDD.scala:204)
	at org.apache.spark.api.python.PythonRDD$$anonfun$writeIte

In [29]:
print(correlation_matrix)

[[ 1.          0.1181646  -0.05113165  0.13488199]
 [ 0.1181646   1.          0.00803336 -0.01342319]
 [-0.05113165  0.00803336  1.         -0.0175606 ]
 [ 0.13488199 -0.01342319 -0.0175606   1.        ]]


In [33]:
from pyspark.mllib.stat import Statistics
import pandas as pd
import numpy as np
# result can be used w/ seaborn's heatmap
def compute_correlation_matrix(df, method='pearson'):
    # wrapper around
    # https://forums.databricks.com/questions/3092/how-to-calculate-correlation-matrix-with-all-colum.html
    df_rdd = df.rdd.map(lambda row: row[0:])
    corr_mat = Statistics.corr(df_rdd, method=method)
    corr_mat_df = pd.DataFrame(corr_mat,
                    columns=df.columns, 
                    index=df.columns)
    return corr_mat_df

In [34]:
jjj=compute_correlation_matrix(df=h_df2)

In [35]:
jjj

,%call drops,total day minutes,total intl calls,customer service calls
%call drops,1.000000,0.118165,-0.051132,0.134882
total day minutes,0.118165,1.000000,0.008033,-0.013423
total intl calls,-0.051132,0.008033,1.000000,-0.017561
customer service calls,0.134882,-0.013423,-0.017561,1.000000
